In [1]:
!pip3 install transformers
!pip3 install pytorch-pretrained-bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 11.3 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from transformers import BertTokenizer
import torch
from torch.utils.data import TensorDataset

In [3]:
df = pd.read_csv('/Tweets.csv')
df.rename(columns={'airline_sentiment':'label'},inplace=True)

df = df[['label','text']]
df.head(2)

,label,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...


In [4]:
possible_labels = df.label.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label]= index
print(label_dict)
df['category'] = df['label'].map(label_dict)

{'neutral': 0, 'positive': 1, 'negative': 2}


In [5]:
df.head(2)

,label,text,category
0,neutral,@VirginAmerica What @dhepburn said.,0
1,positive,@VirginAmerica plus you've added commercials t...,1


In [6]:
def split_data(df, split_ratio=0.8):
   
    # Get the number of rows in the DataFrame
    n = len(df)
    
    # Shuffle the row indices
    shuffled_indices = np.random.permutation(n)
    
    # Calculate the split point
    split_point = int(n * split_ratio)
    
    # Split the DataFrame into two datasets using the shuffled indices
    first_dataset = df.iloc[shuffled_indices[:split_point]]
    second_dataset = df.iloc[shuffled_indices[split_point:]]
    
    return first_dataset, second_dataset

In [7]:
train, test= split_data(df)

In [8]:
df.groupby(['category']).count()

,label,text
category,,
0,3099,3099
1,2363,2363
2,9178,9178


In [9]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [10]:
encoded_data_train = tokenizer.batch_encode_plus(
    train.text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    #pad_to_max_length=True,
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    test.text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    #pad_to_max_length=True,
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors='pt'
)

In [11]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train.category.values) 

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test.category.values) 

In [12]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)
dataset_test = TensorDataset(input_ids_test, 
                              attention_masks_test,
                              labels_test, torch.tensor(test.index))


In [13]:
len(dataset_train)

11712

In [14]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased', 
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = True
                                     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [16]:
batch_size = 32

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_test = DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=32
)

In [17]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [18]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)
epochs = 5

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')


In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)


cuda


In [21]:
from tqdm.notebook import tqdm
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [22]:
from sklearn.metrics import f1_score
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [23]:
from tqdm.notebook import tqdm
outputs={}
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')
    
    tqdm.write('\nEpoch # '+str(epoch))
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Test loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/366 [00:00<?, ?it/s]


Epoch # 1
Training loss: 0.5483350154073512


  0%|          | 0/92 [00:00<?, ?it/s]

Test loss: 0.42616488444416417
F1 Score (weighted): 0.8315891811169716


...

Test loss: 0.448769212619442
F1 Score (weighted): 0.8431295987286473


Epoch 5:   0%|          | 0/366 [00:00<?, ?it/s]


Epoch # 5
Training loss: 0.20719245719453677


  0%|          | 0/92 [00:00<?, ?it/s]

Test loss: 0.4581656362861395
F1 Score (weighted): 0.844349277186682


In [24]:
accuracy_per_class(predictions, true_vals)
print(encoded_data_train.input_ids.shape)
print(len(outputs.hidden_states))
print(outputs.hidden_states[12].shape)

Class: neutral
Accuracy:432/626

Class: positive
Accuracy:373/461

Class: negative
Accuracy:1668/1841

torch.Size([11712, 67])
13
torch.Size([32, 67, 768])


In [25]:
print(len(predictions))
def missed_by_cat(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    missed={}
    
    for label in np.unique(labels_flat):
        indexes=np.where(labels_flat==label)
        indexes=np.where(preds_flat[indexes]!=labels_flat[indexes] )
        missed[label_dict_inverse[label]]=indexes
      
    return missed
misses=missed_by_cat(predictions, true_vals)
print(misses)
print(dataloader_test)
print(test.iloc[misses['neutral'][0][0]])

2928
{'neutral': (array([  6,  14,  15,  20,  22,  26,  29,  30,  32,  34,  40,  42,  43,
        44,  45,  47,  48,  58,  61,  63,  69,  71,  74,  75,  78,  83,
        84,  88,  89,  91,  93,  97, 100, 103, 107, 112, 113, 114, 115,
       116, 117, 119, 120, 121, 122, 131, 137, 138, 141, 145, 147, 148,
       149, 152, 154, 155, 156, 160, 164, 171, 173, 175, 181, 183, 191,
       192, 196, 197, 199, 202, 208, 211, 225, 229, 230, 232, 233, 234,
       240, 241, 243, 246, 250, 257, 262, 263, 266, 268, 269, 270, 279,
       281, 287, 295, 298, 301, 302, 304, 305, 308, 309, 314, 318, 331,
       333, 334, 335, 337, 340, 342, 346, 354, 355, 359, 364, 365, 369,
       372, 375, 376, 379, 384, 387, 390, 393, 399, 401, 403, 404, 407,
       410, 412, 414, 415, 416, 419, 425, 427, 433, 437, 438, 441, 447,
       448, 451, 452, 458, 460, 462, 463, 464, 465, 469, 473, 474, 477,
       479, 483, 485, 486, 496, 504, 509, 512, 513, 516, 520, 526, 528,
       529, 537, 539, 542, 544, 557, 563, 564,

In [26]:
def val(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    indexes=[]
    missed_indexes=[]
    
    for batch in tqdm(dataloader_val):
        batch = tuple(b.to(device) for b in batch)
        
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
        indexes.append(batch[3].cpu().numpy())
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    indexes=np.concatenate(indexes, axis=0)
            
    return loss_val_avg, predictions, true_vals, indexes
loss_val_avg, predictions, true_vals, indexes=val(dataloader_test)


  0%|          | 0/92 [00:00<?, ?it/s]

In [29]:
def missed_b(preds, labels,indexes):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    indexes=indexes.flatten()
    missed={}

    
    for label in np.unique(labels_flat):
        ind=np.where(labels_flat==label)
        misses=[]
        for idx in ind[0]:
          if preds_flat[idx]!=labels_flat[idx]:
            misses.append(idx)
          
        missed[label_dict_inverse[label]]=[indexes[idx] for idx in ind[0]]
      
    return missed
misses=missed_b(predictions, true_vals, indexes)
print(indexes[0])
print(misses['neutral'][0])
print(test.loc[[6562]])

6277
6562
        label                                               text  category
6562  neutral  @SouthwestAir @TomVH just get the RR credit ca...         0


In [37]:
neutral_misses=misses['neutral']
negative_misses=misses['negative']
positive_misses=misses['positive']
missed_vals= test.loc[neutral_misses]
missed_vals.append(test.loc[negative_misses],ignore_index = True)
missed_vals.append(test.loc[positive_misses],ignore_index = True)
missed_vals.to_csv('/misses.csv')


In [ ]:
!pip3 install sentence_transformers
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('stsb-bert-base')

In [ ]:

query = "I hate Spirit"
query_vec = sbert_model.encode([query])[0]
print(query_vec)